In [1]:
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
import math
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def log_transform(error):
    return math.copysign(1.0,error)*math.log(math.fabs(error)+1,2)

In [3]:
def softmax(arr,t=1.0):
    w = np.array(arr)
    e = np.exp(w / t)
    dist = e / np.sum(e)
    return dist

In [4]:
def argmax(arr_3d,outer,inner):
    max_row = outer[0]
    max_col = inner[0]
    max_x = 0
    #max_value = arr_2d[0,0]
    max_value = arr_3d[outer[0],inner[0],0]
    for row in range(arr_3d.shape[0]):
        if row not in outer:
            continue
        for col in range(arr_3d.shape[1]):
            if col not in inner:
                continue
            for x in range(arr_3d.shape[2]):
                if arr_3d[row,col,x] > max_value:
                    max_value = arr_3d[row,col,x]
                    max_row,max_col,max_x = row,col,x
    return list((max_row,max_col,max_x))

In [5]:
def context_check(outer,inner):
    if outer == 0:
        if inner == 0:
            return 'AX'
        elif inner == 1:
            return 'AY'
    elif outer == 1:
        if inner == 0:
            return 'BX'
        elif inner == 1:
            return 'BY'

In [6]:
def performance(outer,inner,action,cont,arr_2d):
    # arr_2d[count,numcorrect,performance]
    if context_check(outer,inner)=='AX':
        #count1+=1
        arr_2d[0,0]+=1
        if action == 0 and cont == 0 or action == 1 and cont == 1:
            #nc1 += 1
            arr_2d[0,1]+=1
        #AX_perf = nc1/count1
        arr_2d[0,2] = arr_2d[0,1]/arr_2d[0,0]
    elif context_check(outer,inner)=='BX':
        #count2+=1
        arr_2d[1,0]+=1
        if action == 1:
            #nc2 += 1
            arr_2d[1,1]+=1
        #BX_perf = nc2/count2
        arr_2d[1,2] = arr_2d[1,1]/arr_2d[1,0]
    elif context_check(outer,inner)=='AY':
        #count3+=1
        arr_2d[2,0]+=1
        if action == 1:
            #nc3 += 1
            arr_2d[2,1]+=1
        #AY_perf = nc3/count3
        arr_2d[2,2] = arr_2d[2,1]/arr_2d[2,0]
    elif context_check(outer,inner)=='BY':
        #count4+=1
        arr_2d[3,0]+=1
        if action == 1 and cont == 0 or action == 0 and cont == 1:
            #nc4 += 1
            arr_2d[3,1]+=1
        #BY_perf = nc4/count4
        arr_2d[3,2] = arr_2d[3,1]/arr_2d[3,0]

In [21]:
def TD(ntrials,lrate,gamma,td_lambda,temp,decay,time):
    n = 1024
    nactions = 2 # number of actions
    ncont_wm = 2 # number of context wm
    nwm_o = 2 # number of outer wm slots
    nwm_i = 2 # number of inner wm slots
    nsig_o = 2 # number of cue signals
    nsig_i = 2 # number of probe signals
    ncont_o = 2 # number of context signals
    
    # reward matix, reward given at 0,0,0
    # A-X target context
    
    reward1 = np.zeros((nsig_o+1,nsig_i+1,nactions))
    reward1[0,0,0] = 1
    reward1[0,1,1] = 1
    reward1[1,0,1] = 1
    reward1[1,1,1] = 1
    # B-Y target context
    reward2 = np.zeros((nsig_o+1,nsig_i+1,nactions))
    reward2[0,0,1] = 1
    reward2[0,1,1] = 1
    reward2[1,0,1] = 1
    reward2[1,1,0] = 1
    
    """
    reward = np.zeros((ncont_o+1,nsig_o+1,nsig_i+1,nactions))
    reward[0,0,0,0] = 1
    reward[0,0,1,1] = 1
    reward[0,1,0,1] = 1
    reward[0,1,1,1] = 1
    
    reward[1,0,0,1] = 1
    reward[1,0,1,1] = 1
    reward[1,1,0,1] = 1
    reward[1,1,1,0] = 1
    """
    #reward_outer = np.zeros((nsig_o+1,nactions))
    #reward_outer[0,0] = 1
    #reward_outer[1,1] = 1
    # hrr for actions
    actions = hrr.hrrs(n,nactions)
    
    # identity vector
    hrr_i = np.zeros(n)
    hrr_i[0] = 1
    
    # context outer cue
    cont_outer = hrr.hrrs(n,ncont_o)
    cont_outer = np.row_stack((cont_outer,hrr_i))
    
    # cue outer
    sig_outer = hrr.hrrs(n,nsig_o)
    sig_outer = np.row_stack((sig_outer,hrr_i))
    
    # probe inner
    sig_inner = hrr.hrrs(n,nsig_i)
    sig_inner = np.row_stack((sig_inner,hrr_i))
     
    # context working memory
    wm_cont = hrr.hrrs(n,ncont_wm)
    wm_cont = np.row_stack((wm_cont,hrr_i))
        
    # outer working memory
    wm_outer = hrr.hrrs(n,nwm_o)
    wm_outer = np.row_stack((wm_outer,hrr_i))
    
    # inner working memory
    wm_inner = hrr.hrrs(n,nwm_i)
    wm_inner = np.row_stack((wm_inner,hrr_i))
    
    # precomputed
    external = hrr.oconvolve(sig_inner,sig_outer) 
    s_a = hrr.oconvolve(external,actions)
    s_a = np.reshape(s_a,(nsig_o+1,nsig_i+1,nactions,n))
    # weight vector and bias
    W = hrr.hrr(n)
    bias = 1
    
    # epsilon for e-soft policy
    epsilon = .01
    
    # temperature for softmax
    t = temp
    
    # eligibility trace
    eligibility = np.zeros(n)
    # array that keeps track of AX-CPT performance
    perf_arr = np.zeros((4,3))
    context = 0
    for trial in range(ntrials):
        r = 0
        eligibility = np.zeros(n)
        # 70% AX trials, 30% AY,BX,BY trials #
        index = np.random.choice([0,1,2,3],p=[.25,.25,.25,.25])
        choices = [(0,0),(0,1),(1,0),(1,1)]
        cue,probe = choices[index]
        #################################
        if trial%200==0:
            context = np.random.choice([0,1],p=[.5,.5])
        #context = 0
            
        if context == 0:
            reward = reward1 # updates reward policy for A-X target
        elif context == 1:
            reward = reward2 # updates reward policy for B-Y target
        # sets context for later use
    
        outer = cue
        
        inner = probe
        
        ####### stage 1
        cont_outerwm = hrr.convolve(cont_outer[context], wm_cont)
        values = np.dot(cont_outerwm,W) + bias
        possible_context = np.unique([context])
        
        sm_prob = softmax(values,t)
        #print(sm_prob)
        wm_o = np.unravel_index(np.argmax(np.take(sm_prob,possible_context)),sm_prob.shape)
        wm_o = possible_context[wm_o]
        
        trace1 = hrr.convolve(sig_outer[context],wm_cont[wm_o])
        wm1 = wm_cont[wm_o]
        
        pvalues = values[wm_o] # stores previous Q value
        eligibility = cont_outerwm[wm_o] + td_lambda*eligibility
        ##### end stage 1
        
        
        ###### is cue worth remembering #######
        #######################################
        cue_outerwm = hrr.convolve(sig_outer[cue],wm_outer)
        values = np.dot(cue_outerwm,W) + bias
        possible_context = np.unique([cue,2])
        
        sm_prob = softmax(values,t)
        wm_o = np.unravel_index(np.argmax(np.take(sm_prob,possible_context)),sm_prob.shape)
        wm_o = possible_context[wm_o]
        #wm1 = wm_outer[wm_o] # selected memory slot
        #print('convolve:',cue_outerwm.shape)
        #print('values:',values.shape)
        #print('smax:',sm_prob.shape)
        #print('wm1:',wm1.shape)
        #print(wm_o)
        #print(sm_prob)
        value = values[wm_o]
        ########## epsilon soft ##################
        if random.random() < epsilon:
            wm_o = random.randrange(nwm_o+1)
            
        #error = (r+gamma*value)-pvalues
        #W += lrate*log_transform(error)*eligibility
        
        trace1 = hrr.convolve(sig_outer[cue],wm_outer[wm_o])
        wm1 = wm_outer[wm_o] # selected memory slot
        ###### decay chosen workingMemory ########
        wm_outer_decayed = np.array(wm1)
        wm_outer_decayed = hrr.pow(wm_outer_decayed,decay**time)
        #trace1 = hrr.convolve(sig_outer[cue],wm_outer_decayed) #changed
        #######################################
        
        #r = 0 # reward for outer memory choice
        pvalue = values[wm_o] # stores previous Q value
        eligibility = cue_outerwm[wm_o] + td_lambda*eligibility
        #eligibility = trace1 + td_lambda*eligibility
        #######################################
        wm1_wm2 = hrr.convolve(wm_outer_decayed,wm_inner) # convolve chosen outer wm with matrix of inner wm choices
        #wm1_wm2 = hrr.convolve(trace1,wm_inner) # changed
        probe_outerinnerwm_a = hrr.convolve(sig_inner[probe],hrr.oconvolve(wm1_wm2,actions))
        probe_outerinnerwm_a = np.reshape(probe_outerinnerwm_a,(nwm_o+1,nactions,n))
        values = np.dot(probe_outerinnerwm_a,W) + bias
        possible_context = np.unique([probe,2])
        
        sm_prob = softmax(values,t)
        wm_i = np.unravel_index(np.argmax(sm_prob),sm_prob.shape) 
        current_memory = wm_i[0]
        action = wm_i[1]
        
        #print('probe:',probe_outerinnerwm_a.shape)
        #print('value:',values.shape)
        #print('smax:',sm_prob.shape)
        #print(sm_prob)
        #print(eligibility.shape)
        #######################################
    
        ######### epsilon soft policy ##########
        if random.random() < epsilon:
            
            action = random.randrange(0,nactions)
            current_memory = random.randrange(nwm_i+1)
            #current_outer_wm = random.randrange(nwm_o+1)
            #current_inner_wm = random.randrange(nwm_i+1)
        ########################################  
        r = reward[outer,inner,action]
        #print(outer,inner,action,"\t",r)
        value = values[current_memory,action]
        error = (r + gamma*value) - pvalue
        W += lrate*log_transform(error)*eligibility
        
        eligibility = probe_outerinnerwm_a[current_memory,action,:] + td_lambda*eligibility
        ########################################
        value = values[current_memory,action]
        #r = reward[outer,inner,action]
        error = r - value
        W += lrate*log_transform(error)*eligibility
        performance(outer,inner,action,context,perf_arr)
        ########################################
        #print(context,cue,probe,action)
        if trial%1000==0:
            print('Trial:',trial,end='\n\n')
            print(format('','>10s'),format('count','>12s'),format('performance','>20s'))
            print(format('AX |','<10s'),format(perf_arr[0,0],'>12.1f'),format(perf_arr[0,2],'>20.2%'))
            print(format('BX |','<10s'),format(perf_arr[1,0],'>12.1f'),format(perf_arr[1,2],'>20.2%'))
            print(format('AY |','<10s'),format(perf_arr[2,0],'>12.1f'),format(perf_arr[2,2],'>20.2%'))
            print(format('BY |','<10s'),format(perf_arr[3,0],'>12.1f'),format(perf_arr[3,2],'>20.2%'))
            perf_arr = np.zeros((4,3))
            print(end='\n\n')
        
    print('Trial:',trial,end='\n\n')
    print(format('','>10s'),format('count','>12s'),format('performance','>20s'))
    print(format('AX |','<10s'),format(perf_arr[0,0],'>12.1f'),format(perf_arr[0,2],'>20.2%'))
    print(format('BX |','<10s'),format(perf_arr[1,0],'>12.1f'),format(perf_arr[1,2],'>20.2%'))
    print(format('AY |','<10s'),format(perf_arr[2,0],'>12.1f'),format(perf_arr[2,2],'>20.2%'))
    print(format('BY |','<10s'),format(perf_arr[3,0],'>12.1f'),format(perf_arr[3,2],'>20.2%'))
    perf_arr = np.zeros((4,3))
    print(end='\n\n')
         

In [23]:
TD(20000,.1,.9,.8,.1,0,0)
# (num trials, learning rate, discount factor, lambda, temperature, decay factor, decay time steps)

Trial: 0

                  count          performance
AX |                1.0                0.00%
BX |                0.0                0.00%
AY |                0.0                0.00%
BY |                0.0                0.00%


Trial: 1000

                  count          performance
AX |              247.0               94.74%
BX |              260.0               98.08%
AY |              260.0               97.31%
BY |              233.0               95.28%


Trial: 2000

                  count          performance
AX |              256.0               96.88%
BX |              257.0               99.22%
AY |              231.0               98.27%
BY |              256.0               94.14%


Trial: 3000

                  count          performance
AX |              258.0               98.84%
BX |              235.0               99.15%
AY |              256.0               98.44%
BY |              251.0               96.02%


Trial: 4000

                  count       

In [60]:
x = np.array([1,2,3])
print(x[None,1].shape)

(1,)


In [64]:
np.argmax(np.take(x,[0,2]))
#mask = [True,True,False]

1

In [40]:
np.where(x>2,2,1)

array([1, 1, 2])

In [41]:
x[mask]

array([1, 2])

In [146]:
count = 12
perf = .45
print(format('','>20s'),format('count','>12s'),format('performance','>20s'))
print(format('AX |','<20s'),format(count,'>12d'),format(perf,'>20%'))
print(format('AY |','<20s'),format(count,'>12d'),format(perf,'>20%'))
print(format('BX |','<20s'),format(count,'>12d'),format(perf,'>20%'))
print(format('BY |','<20s'),format(count,'>12d'),format(perf,'>20%'))

                            count          performance
AX |                           12           45.000000%
AY |                           12           45.000000%
BX |                           12           45.000000%
BY |                           12           45.000000%
